In [19]:
import torch.nn as nn
import torch.functional as F
import torch
import transformers
import os
import numpy as np
from transformers import convert_bert_original_tf_checkpoint_to_pytorch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, BertForQuestionAnswering, BertForPreTraining

In [25]:
BERT_MODEL_PATH = "rubert/"
os.listdir(BERT_MODEL_PATH)

['.DS_Store',
 'bert_model.ckpt.data-00000-of-00001',
 'bert_config.json',
 'vocab.txt',
 '.ipynb_checkpoints',
 'bert_model.ckpt.index']

In [31]:
convert_bert_original_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
    BERT_MODEL_PATH + 'bert_model.ckpt',
BERT_MODEL_PATH + 'bert_config.json','pytorch_model.bin')

Building PyTorch model from configuration: {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 119547
}



INFO:transformers.modeling_bert:Converting TensorFlow checkpoint from /Users/potemin/work/rustory_generation/rubert/bert_model.ckpt
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/LayerNorm/beta with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/LayerNorm/gamma with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/position_embeddings with shape [512, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/token_type_embeddings with shape [2, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/embeddings/word_embeddings with shape [119547, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_0/attention/output/LayerNorm/gamma with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_0/attention/output/dense/bias with sha

INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/output/LayerNorm/gamma with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/output/dense/bias with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/output/dense/kernel with shape [768, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/self/key/bias with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/self/key/kernel with shape [768, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/self/query/bias with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/self/query/kernel with shape [768, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_2/attention/self/value/bias with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encod

INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/attention/self/value/bias with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/attention/self/value/kernel with shape [768, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/intermediate/dense/bias with shape [3072]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/intermediate/dense/kernel with shape [768, 3072]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/output/LayerNorm/beta with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/output/LayerNorm/gamma with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/output/dense/bias with shape [768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_6/output/dense/kernel with shape [3072, 768]
INFO:transformers.modeling_bert:Loading TF weight bert/encoder/layer_7/attention/output/Lay

INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'output', 'dense', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'key', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'key', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'query', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'query', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'value', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'value', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'intermediate', 

INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'attention', 'self', 'key', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'attention', 'self', 'key', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'attention', 'self', 'query', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'attention', 'self', 'query', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'attention', 'self', 'value', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'attention', 'self', 'value', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'intermediate', 'dense', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_2', 'intermediate', 'dense', 

INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'query', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'query', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'value', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'attention', 'self', 'value', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'intermediate', 'dense', 'bias']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'intermediate', 'dense', 'kernel']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'output', 'LayerNorm', 'beta']
INFO:transformers.modeling_bert:Initialize PyTorch weight ['bert', 'encoder', 'layer_6', 'output', 'LayerNorm', 'gamma']
I

Save PyTorch model to pytorch_model.bin


In [21]:
model_path = "rubert_torch/"

In [22]:
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForMaskedLM.from_pretrained(model_path)

INFO:transformers.tokenization_utils:Model name 'rubert_torch/' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc). Assuming 'rubert_torch/' is a path or url to a directory containing tokenizer files.
INFO:transformers.tokenization_utils:Didn't find file rubert_torch/added_tokens.json. We won't load it.
INFO:transformers.tokenization_utils:Didn't find file rubert_torch/special_tokens_map.json. We won't load it.
INFO:transformers.tokenization_utils:Didn't find file rubert_torch/tokenizer_config.json. We won't load it.
INFO:transformers.tokenization_utils:loading file rubert_torch/vocab.txt
INFO:transformers.tok

In [24]:
model.parameters

<bound method Module.parameters of BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768